異体字のペアを作成する

In [1]:
%cd itaiji

/home/analysis01/src/kanjikana-model/dict/itaiji


ペアにして、属性をつける。 

key1,key2:[attr1,attr2]
key2,key1:[attr1,attr2]      

In [2]:
pair={}

In [3]:
def make_pair(fname,attr):
    with open(fname,'r',encoding='utf-8') as f:
        for l in f:
            items=l.rstrip().split(',')
            keyA=f'{items[0]},{items[1]}'      
            keyB=f'{items[1]},{items[0]}'
            if keyA not in pair:
                pair[keyA]=[]
            if keyB not in pair:
                pair[keyB]=[]
            pair[keyA].append(attr)
            alst=pair[keyA]
            pair[keyB].append(attr)
            blst=pair[keyB]
            pair[keyA]=list(set(alst))
            pair[keyB]=list(set(blst))

In [4]:
make_pair('../oss/mozc/itaiji.txt','mozc_itaiji')

In [5]:
len(pair)

1420

In [6]:
make_pair('../oss/skk/itaiji.txt','skk_itaiji')

In [7]:
len(pair)

2344

In [8]:
siz={}
for k,v in pair.items():
    if len(v) not in siz:
        siz[len(v)]=0
    siz[len(v)]+=1

In [9]:
siz

{1: 1112, 2: 1232}

同じ文字のグループを作成する

In [10]:
group={}  # group[kanji1]=groupid    漢字単体とグループ番号を作成

In [11]:
for k in pair.keys():
    key1,key2=k.split(',')
    group[key1]=-1
    group[key2]=-1

In [12]:
gid=0

In [13]:
for k in pair.keys():
    key1,key2=k.split(',')
    cand_gid=group[key1]
    if cand_gid<0:
        group[key2]
    if cand_gid<0: # newgroup
        group[key1]=gid
        group[key2]=gid
        gid+=1
    else:
        group[key1]=cand_gid
        group[key2]=cand_gid

In [14]:
gid

845

In [15]:
rev_group={}   # group[groupid]=[kanji1,kanji2,]    漢字単体とグループ番号を作成

In [16]:
for k,v in group.items():
    if v not in rev_group:
        rev_group[v]=[]
    rev_group[v].append(k)

In [17]:
len(rev_group)

791

In [18]:
rev_group

{0: ['叱', '𠮟'],
 1: ['埓', '埒'],
 2: ['髙', '高'],
 3: ['啮', '噛', '嚙'],
 4: ['噓', '嘘'],
 5: ['蟬', '蝉'],
 6: ['驒', '騨'],
 7: ['簞', '箪'],
 8: ['剝', '剥'],
 9: ['屛', '屏'],
 10: ['穎', '頴'],
 11: ['禰', '袮'],
 743: ['祢'],
 12: ['儘', '侭'],
 13: ['壺', '壷'],
 228: ['彎', '弯', '灣', '湾'],
 15: ['攪', '撹'],
 16: ['檜', '桧'],
 17: ['檮', '梼'],
 18: ['櫟', '檪'],
 19: ['濤', '涛'],
 20: ['灌', '濯'],
 662: ['潅'],
 21: ['濾', '沪', '瀘'],
 22: ['箏', '筝'],
 23: ['籠', '篭'],
 375: ['藪'],
 24: ['薮', '籔'],
 25: ['蘆', '芦'],
 26: ['蠣', '蛎'],
 27: ['蠅', '蝿'],
 28: ['諫', '諌'],
 29: ['賤', '賎'],
 30: ['邇', '迩'],
 31: ['靱', '靭', '靫'],
 32: ['頸', '頚'],
 33: ['鰺', '鯵'],
 34: ['鶯', '鴬'],
 35: ['俠', '侠'],
 36: ['啞', '唖'],
 37: ['囊', '嚢'],
 38: ['塡', '填'],
 39: ['搔', '掻'],
 40: ['潑'],
 244: ['溌', '醗', '發', '発'],
 41: ['瀆', '涜'],
 42: ['焰', '焔'],
 43: ['禱', '祷'],
 44: ['繡'],
 819: ['繍', '綉'],
 45: ['繫', '繋'],
 46: ['萊', '莱'],
 47: ['蔣', '蒋'],
 48: ['蠟', '蝋'],
 49: ['軀', '躯'],
 50: ['醬', '醤'],
 51: ['醱'],
 52: ['頰', '頬'],
 53: ['顚', '顛

第一水準から順に優先順をつけて、優先のものに代替するようにする
さらに小中高の順に優先する

In [19]:
def load_jis(fname):
    with open(fname,'r',encoding='utf-8') as f:
        hsh ={l.rstrip():1 for l in f}
    return hsh

In [20]:
import regex
import re

In [21]:
def get_suijun(s):
    # 平仮名カタカナは最優先 0でかえす  旧字は除外
    if (regex.match('^\p{Script=Hiragana}+$',s) is not None or regex.match('^\p{Script=Katakana}+$',s) is not None) and re.match(f'^[ゐゑ	゚゛゜ゝゞゟ]',s) is None and re.match('^[ヰヱヸヹヽヾヿ]+$',s) is None:
        return 0 
    return 1

In [22]:
def get_min_suijun(lst): # 配列内で最も水準が高いものの水準
    min_suijun=5
    min_moji=''
    for l in lst:
        sui=get_suijun(l)
        if min_suijun>sui:
            min_suijun=sui
            min_moji=l
    if min_suijun==5:
        min_moji=lst[0]  # 全部1ー4水準外の時は最初のものを選択
    return min_suijun,min_moji        

In [23]:
ngroup={}  # キーが水準の低いもののみ残す  {key2:key1,key3:key1,key4:key1}  key1 is min suijun

In [24]:
for vv in rev_group.values():
    min_sui,min_moji=get_min_suijun(vv)
    for val in vv:
        if val == min_moji:
            continue
        key=f'{min_moji},{val}'
        attr=[]
        if key in pair:
            attr=list(set(pair[key]))
        ngroup[val]={min_moji:{"dics":attr}}


In [25]:
len(ngroup)

896

In [26]:
ngroup

{'𠮟': {'叱': {'dics': ['mozc_itaiji']}},
 '埒': {'埓': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '高': {'髙': {'dics': ['mozc_itaiji']}},
 '噛': {'啮': {'dics': ['mozc_itaiji']}},
 '嚙': {'啮': {'dics': []}},
 '嘘': {'噓': {'dics': ['mozc_itaiji']}},
 '蝉': {'蟬': {'dics': ['mozc_itaiji']}},
 '騨': {'驒': {'dics': ['mozc_itaiji']}},
 '箪': {'簞': {'dics': ['mozc_itaiji']}},
 '剥': {'剝': {'dics': ['mozc_itaiji']}},
 '屏': {'屛': {'dics': ['mozc_itaiji']}},
 '頴': {'穎': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '袮': {'禰': {'dics': ['skk_itaiji']}},
 '侭': {'儘': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '壷': {'壺': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '弯': {'彎': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '灣': {'彎': {'dics': ['skk_itaiji']}},
 '湾': {'彎': {'dics': ['skk_itaiji']}},
 '撹': {'攪': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '桧': {'檜': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '梼': {'檮': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '檪': {'櫟': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '涛': {'濤'

In [27]:
import json

In [28]:
with open('../itaiji.json','w',encoding='utf-8') as f:
    json.dump(ngroup,f,ensure_ascii=False,indent=2)

In [29]:
!date

2025年  5月 30日 金曜日 14:28:25 JST
